In [35]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import glob

In [36]:
data = {}

for path in glob.glob("/home/eagle/ownCloud/ma_thesis/results/*.json"):
  for typename, items in json.load(open(path)).items():
    if typename in data:
      data[typename].update(items)
    else:
      data[typename] = items

frames = { typename: pd.DataFrame.from_dict(items, orient='index') for typename, items in data.items() }

In [47]:
frames['algorithm_level'] \
    .loc[lambda x: x.level == 0] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'].loc[lambda x: x.graph.str.contains("amazon")], left_on='program_run_id', right_index=True) \
    .loc[lambda x: x.algorithm.str.contains("sequential")] \
    ['cluster_count'] \
    .mean()

52814.86666666667

In [48]:
frames['algorithm_level'] \
    .loc[lambda x: x.level == 0] \
    .merge(frames['algorithm_run'], left_on='algorithm_run_id', right_index=True) \
    .merge(frames['program_run'].loc[lambda x: x.graph.str.contains("amazon")], left_on='program_run_id', right_index=True) \
    .loc[lambda x: x.algorithm.str.contains("partitioned")] \
    .groupby(['order', 'partition_algorithm', 'partition_count'])['cluster_count'].mean()

order             partition_algorithm                       partition_count
clustering_based  chunk                                     4.0                 61212
                                                            32.0                83657
                                                            128.0              152496
                                                            1024.0             270110
                  deterministic_greedy_with_linear_penalty  4.0                132871
                                                            32.0               262791
                                                            128.0              291156
                                                            1024.0             296293
original          chunk                                     4.0                222592
                                                            32.0               314020
                                                            128.

In [49]:
frames['program_run'].loc[lambda x: x.graph.str.contains("amazon")]['node_count']

ac2940fa-afc8-42ff-8c45-d58304ce9ee7    334863
b8ab11eb-d30b-4266-a924-b5b6bcf38fc3    334863
c4501056-9236-4a10-b4e4-0d642652456f    334863
da63d7de-6257-4f85-bc79-7f6a48e6c39c    334863
fd9a93a7-044d-477f-a750-4425c1f518ed    334863
Name: node_count, dtype: int64